In [1]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
import re

import time
from tqdm import tqdm_notebook

import json

In [33]:
url = 'https://en.wikipedia.org/wiki/List_of_regions_of_Canada'
df_can = pd.read_html(url, attrs={"class": "wikitable"})[0] # 0 is for the 1st table in this particular page
canada = list(df_can.iloc[:,0])

In [35]:
url = 'https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States'
df_usa = pd.read_html(url, attrs={"class": "wikitable"})[0] # 0 is for the 1st table in this particular page
usa = list(df_usa.iloc[:,0].apply(lambda x : x.replace("[E]","").replace("[F]","")))

In [36]:
url = 'https://en.wikipedia.org/wiki/Countries_of_the_United_Kingdom'
df_uk = pd.read_html(url, attrs={"class": "wikitable"})[0] # 0 is for the 1st table in this particular page
uk = list(df_uk.iloc[:-1,0])

In [37]:
url = 'https://en.wikipedia.org/wiki/States_and_territories_of_Australia'
df_aus = pd.read_html(url, attrs={"class": "wikitable"})[0] # 0 is for the 1st table in this particular page
australia = list(df_aus[df_aus.iloc[:,1] == 'Federated state'].iloc[:,0])

In [40]:
url_can = f'https://ca.indeed.com/jobs?'
url_usa = f'https://www.indeed.com/jobs?'
url_uk = f'https://www.indeed.co.uk/jobs?'
url_aus = f'https://au.indeed.com/jobs?'

In [113]:
canada

['British Columbia',
 'Alberta',
 'Saskatchewan',
 'Manitoba',
 'Ontario',
 'Quebec',
 'New Brunswick',
 'Prince Edward Island',
 'Nova Scotia',
 'Newfoundland and Labrador',
 'Yukon',
 'Northwest Territories',
 'Nunavut']

In [68]:
states = [canada,usa,uk,australia]
countries = ["Canada","United States","United Kingdom","Australia"]
links = [url_can,url_usa,url_uk,url_aus]

country_dict = {}

for country,state,link in zip(countries,states,links):
    country_dict[country] = [state]
    country_dict[country].append(link)

In [72]:
country_dict['Canada'][1]

'https://ca.indeed.com/jobs?'

In [73]:
list_of_searchwords = ['data+scientist','data+analyst','research+scientist','business+intelligence','machine+learning']

In [111]:
#Determining number of jobs per state,region, area etc

global_jobs = {'country':[],'state':[],'role':[],'jobs':[]}

for country in tqdm_notebook(countries):
    print(country)    

    for state in tqdm_notebook(country_dict[country][0]):
        print(state)

        for role in list_of_searchwords:

            count_url = country_dict[country][1]+f'as_ttl={role}&jt=all&radius=5000&l={state}&fromage=any&limit=50&filter=0'
            time.sleep(np.random.randint(0,2))

            counter = requests.get(count_url)
            soup_count = BeautifulSoup(counter.text,'html.parser')
            

            if len(soup_count.find_all('div',class_='bad_query'))==1:
                #print("bad query")

                pass

            else:

                try:
                    record_string = soup_count.find_all('div',attrs={'id':"searchCountPages"})[0].text
                    record_string = record_string.replace(",","")
                    max_results_per_city = int(re.search(r"(\w+)\sjobs",record_string).group(1))
                    #print(max_results_per_city)

                except:
                    max_results_per_city = np.nan
                    #print("NAN")

                #try:
                    #true_pages = int(soup_count.find_all("span",class_="pn")[-2].text)
                    #print(true_pages,max_results_per_city)

                #except:
                    #true_pages = 0
                    #print(true_pages)

                global_jobs['country'].append(country)
                global_jobs['state'].append(state)
                global_jobs['role'].append(role)
                global_jobs['jobs'].append(max_results_per_city)   

Canada


British Columbia
Alberta
Saskatchewan
Manitoba
Ontario
Quebec
New Brunswick
Prince Edward Island
Nova Scotia
Newfoundland and Labrador
Yukon
Northwest Territories
Nunavut
United States


Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming
United Kingdom


England
Northern Ireland
Scotland
Wales
Australia


South Australia
Tasmania
New South Wales
Victoria
Western Australia
Queensland


In [112]:
job_nums = pd.DataFrame(global_jobs).sort_values(by='jobs',ascending=False)

country_searchlist = job_nums.groupby(["country"]).mean().sort_values(by="jobs",ascending=True).index.to_list()
#state_searchlist= job_nums.groupby(["state"]).mean().sort_values(by="jobs",ascending=True).index.to_list()
#role_searchlist = job_nums.groupby(["role"]).mean().sort_values(by="jobs",ascending=True).index.to_list()
job_nums.head()

,country,state,role,jobs
269,United Kingdom,England,data+analyst,1612
52,United States,California,data+scientist,938
53,United States,California,data+analyst,935
268,United Kingdom,England,data+scientist,914
56,United States,California,machine+learning,708


In [ ]:
df_dict = {'job_title':[],'job_category':[],
           'company':[],'location':[],
           'region':[],'salary':[],
           'link':[],'summary':[]}

#url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 294 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.

all_files = []

for state in tqdm_notebook(state_searchlist):

    for role in tqdm_notebook(role_searchlist):
        
        count_url = f'https://au.indeed.com/jobs?as_ttl={role}&jt=all&radius=5000&l={state}&fromage=any&limit=50&start=0&filter=0'

        time.sleep(np.random.randint(0,1))

        counter = requests.get(count_url)
        soup_count = BeautifulSoup(counter.text,'html.parser')
        
        if len(soup_count.find_all('div',class_='bad_query'))==1:
            
            pass
        
        else:     

            try:
                record_string = soup_count.find_all('div',attrs={'id':"searchCountPages"})[0].text
                record_string = record_string.replace(",","")
                max_results_per_city = int(re.search(r"(\w+)\sjobs",record_string).group(1))
            except:
                max_results_per_city = 1

            for start in tqdm_notebook(range(0, max_results_per_city,50)):
                time.sleep(np.random.randint(0,1))

                URL = f'https://au.indeed.com/jobs?as_ttl={role}&jt=all&radius=5000&l={state}&fromage=any&limit=50&start={start}&filter=0'

                results = requests.get(URL)
                soup = BeautifulSoup(results.text,'html.parser')

                #print(city,int(max_results_per_city))

                #print(state,start,soup.find_all('div',attrs={'id':"searchCountPages"})[0].text)

                if len(soup.find_all('span',attrs={'class':'salary no-wrap'})) == 0:

                    pass

                else:
                    for jobtile in soup.find_all('div',attrs={'class':re.compile(r'^jobsearch-Serp.*unified.*')}):

                        if len(jobtile.find_all('span',attrs={'class':'salary no-wrap'})) == 0:
                            pass
                        else:
                            all_files.append(jobtile)
                            df_dict['job_category'].append(" ".join(role.split("+")).title())
                            df_dict['region'].append(" ".join(state.split("+")).title())

                            detail_link = "https://au.indeed.com/"+extract_href_from_result(jobtile)
                            df_dict['link'].append(detail_link)
                            time.sleep(np.random.randint(0,1))
                            detailgrab = requests.get(detail_link)
                            soup_detail = BeautifulSoup(detailgrab.text,'html.parser')

                            try:
                                df_dict['summary'].append(extract_summary_from_result(soup_detail).text)
                            except:
                                df_dict['summary'].append(np.nan)

In [ ]:
for x in all_files:
    try:
        df_dict['location'].append(extract_location_from_result(x))
    except:
        df_dict['location'].append(np.nan)
        
    try:
        df_dict['company'].append(extract_company_from_result(x))
    except:
        df_dict['company'].append(np.nan)
        
    try:
        df_dict['salary'].append(extract_salary_from_result(x))
        
    except:
        df_dict['salary'].append(np.nan)

    try:
        df_dict['job_title'].append(extract_jobtitle_from_result(x))
        
    except:
        df_dict['job_title'].append(np.nan)

In [ ]:
df = pd.DataFrame(df_dict)

np.sum(df[df.salary.notna()].duplicated() ==False)*100/df.shape[0]

#df = df[df.salary.notna()]

#df = df[df.duplicated()==False]

df.to_csv("aus_jobs.csv",index=False)

In [ ]:
df[df.duplicated()==False]